In [1]:
import pandas as pd
import sklearn

In [3]:
dfTrial = pd.read_csv('../data/trialPromoResults.csv', index_col='index')
dfCust = pd.read_csv('../data/custdatabase.csv', index_col='index')
dfActual = pd.read_csv('../data/Cust_Actual.csv', index_col='index')

In [16]:
dfTrial.head()

,sex,mstatus,age,children,occupation,education,income,avbal,avtrans,decision
index,,,,,,,,,,
1,F,married,56.82,1,legal,secondary,3105.39,33003.48,1776.81,None
2,M,widowed,87.35,3,retired,tertiary,4874.08,18941.99,863.56,None
3,M,single,28.75,0,manuf,professional,14232.37,30013.32,3231.14,B
4,F,married,35.71,0,education,postgrad,3214.93,15423.24,1996.09,None
5,M,single,20.53,0,construct,tertiary,3214.93,15423.24,1996.09,None


In [17]:
dfCust.head()

,sex,mstatus,age,children,occupation,education,income,avbal,avtrans,Unnamed: 10
index,,,,,,,,,,
1001,M,divorced,44.27,2,legal,postgrad,10245.73,53183.04,4069.67,
1002,M,married,61.90,1,legal,secondary,1942.57,18100.78,1522.42,NaN
1003,M,divorced,37.30,1,IT,tertiary,9896.16,24496.82,2723.35,NaN
1004,F,divorced,25.02,1,legal,postgrad,10136.44,26690.01,4143.04,NaN
1005,F,divorced,48.37,2,government,secondary,2521.11,13439.81,240.26,NaN


In [15]:
dfActual.head()

,status,cust Investment Potential Score
index,,
1001,None,5.2
1002,None,2.5
1003,None,4.1
1004,None,4.4
1005,None,2.3
